In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import math

# dimensions of our images.
img_width, img_height = 300, 300

train_data_dir = 'data/train'
validation_data_dir = 'data/test'
nb_train_samples = 3000
nb_validation_samples = 300
epochs = 5
batch_size = 300

Using TensorFlow backend.


In [2]:
input_shape = (img_height, img_width, 3)

## Adding convolutional layers

In [3]:
model = Sequential()
model.add(Conv2D(10, (3, 3), activation = 'relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(20, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(32, activation = 'relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 298, 298, 10)      280       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 149, 149, 10)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 147, 147, 20)      1820      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 73, 73, 20)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 106580)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                3410592   
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
__________

In [5]:
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [6]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 3000 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [7]:
model.fit_generator(
    train_generator,
    steps_per_epoch= math.ceil(nb_train_samples / batch_size),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=1)

Epoch 1/5
10/10 [==============================] - 61s - loss: 1.1534 - acc: 0.4970 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 2/5
10/10 [==============================] - 57s - loss: 0.6919 - acc: 0.5320 - val_loss: 0.6892 - val_acc: 0.6100

### More than 60% now.

### Let's add more convolutions and some dropout/batch normalisation.

In [ ]:
from quiver_engine import server
server.launch(model,  input_folder='./data/test/dogs')

Starting webserver from: /anaconda/envs/py35/lib/python3.5/site-packages/quiver_engine


::ffff:60.241.45.37 - - [2017-09-10 05:07:44] "GET /model HTTP/1.1" 200 4780 0.002506
::ffff:60.241.45.37 - - [2017-09-10 05:07:44] "GET /inputs HTTP/1.1" 200 2980 0.000894
::ffff:60.241.45.37 - - [2017-09-10 05:07:45] "GET /input-file/dog.4757.jpg HTTP/1.1" 200 33208 0.000788
::ffff:60.241.45.37 - - [2017-09-10 05:07:45] "GET /input-file/dog.4571.jpg HTTP/1.1" 200 34545 0.000764
::ffff:60.241.45.37 - - [2017-09-10 05:07:45] "GET /input-file/dog.4930.jpg HTTP/1.1" 200 42671 0.206326
::ffff:60.241.45.37 - - [2017-09-10 05:07:45] "GET /input-file/dog.4677.jpg HTTP/1.1" 200 65093 0.207813
::ffff:60.241.45.37 - - [2017-09-10 05:07:45] "GET /input-file/dog.4697.jpg HTTP/1.1" 200 53641 0.210361
::ffff:60.241.45.37 - - [2017-09-10 05:07:45] "GET /input-file/dog.4563.jpg HTTP/1.1" 200 50804 0.225656
::ffff:60.241.45.37 - - [2017-09-10 05:07:45] "GET /input-file/dog.4867.jpg HTTP/1.1" 200 45026 0.001328
::ffff:60.241.45.37 - - [2017-09-10 05:07:45] "GET /input-file/dog.4490.jpg HTTP/1.1" 200 41